In [1]:
%matplotlib inline

import os, shutil
import agate
from datetime import datetime
import pyproj
from decimal import *

In [2]:
files = ["January2017","February2017","March2017","April2017","May2017","June2017","July2017","August2017","September2017","October2017","November2017","December2017","January2018","February2018","March2018","April2018","May2018","June2018","July2018","August2018","September2018","October2018","November2018","December2018","January2019","February2019","March2019","April2019","May2019"]

## Import files with correct encodings

Some fail by importing as UTF-8

In [3]:
if os.path.exists('temp') and os.path.isdir('temp'):
    shutil.rmtree('temp')

os.mkdir('temp')

for file in files:
    infile_name = "originals/" + file + ".CSV"
    with open(infile_name, mode='r', encoding='cp1252') as infile:
        s = infile.read()
    outfile_name = "temp/" + file + ".csv"
    with open(outfile_name, mode='w', encoding='utf-8') as outfile:
        outfile.write(s)

## Import the files to agate

Let's take a look at the January 2017 table.

In [4]:
print(agate.Table.from_csv('temp/January2017.csv'))

| column             | data_type |
| ------------------ | --------- |
| ï»¿Complaint       | Text      |
| CodedMonth         | Text      |
| DateOccur          | DateTime  |
| FlagCrime          | Boolean   |
| FlagUnfounded      | Boolean   |
| FlagAdministrative | Boolean   |
| Count              | Number    |
| FlagCleanup        | Boolean   |
| Crime              | Number    |
| District           | Number    |
| Description        | Text      |
| ILEADSAddress      | Number    |
| ILEADSStreet       | Text      |
| Neighborhood       | Number    |
| LocationName       | Text      |
| LocationComment    | Text      |
| CADAddress         | Number    |
| CADStreet          | Text      |
| XCoord             | Number    |
| YCoord             | Number    |



Other than the encoding error at the beginning of the first column name, and some of the data types, this looks pretty close to what we want. Let's go ahead and create a function to import tables like this — we'll standardize the column names and the data types.

In [5]:
text_type = agate.Text()
number_type = agate.Number()
boolean_type = agate.Boolean()
datetime_type = agate.DateTime()
date_type = agate.Date()


def import_base_table(file):
    column_names = ['Complaint','CodedMonth','DateOccur','FlagCrime','FlagUnfounded','FlagAdministrative','Count','FlagCleanup','Crime','District','Description','ILEADSAddress','ILEADSStreet','Neighborhood','LocationName','LocationComment','CADAddress','CADStreet','XCoord','YCoord']
    column_types = [text_type,text_type,datetime_type,boolean_type,boolean_type,boolean_type,number_type,boolean_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,number_type,number_type]

    new_table = agate.Table.from_csv(file,column_names,column_types)
    return new_table

First we'll import the base table of January 2017.

In [6]:
jan2017 = import_base_table('temp/January2017.csv')

print(jan2017.column_names)

('Complaint', 'CodedMonth', 'DateOccur', 'FlagCrime', 'FlagUnfounded', 'FlagAdministrative', 'Count', 'FlagCleanup', 'Crime', 'District', 'Description', 'ILEADSAddress', 'ILEADSStreet', 'Neighborhood', 'LocationName', 'LocationComment', 'CADAddress', 'CADStreet', 'XCoord', 'YCoord')


Now what we want to do is check out all our other files. We're testing for two things here: First, that the column names for the table match the base table. Second, that the `CodedMonth` column matches the filename (just to make sure we don't have a file called "December18" with data from December 2017, for example).

Any discrepancies we'll want to check out.

In [7]:
for file in files:
    filename = 'temp/' + file + '.csv'
    cur_table = agate.Table.from_csv(filename)
    if cur_table.column_names == jan2017.column_names:
        print('{} matches base layout'.format(file))
    else:
        print('{} does not match base layout'.format(file))
    if 'CodedMonth' in cur_table.column_names:
        print('CodedMonth(s) for {}:'.format(file))
        for row in cur_table.select(['CodedMonth']).distinct('CodedMonth').rows:
            print(row[0])
    else:
        print('Can\'t find CodedMonth column for {}! Please check manually.'.format(file))
    print('---')

    

January2017 does not match base layout
CodedMonth(s) for January2017:
2017-01
---
February2017 matches base layout
CodedMonth(s) for February2017:
2017-02
---
March2017 matches base layout
CodedMonth(s) for March2017:
2017-03
---
April2017 matches base layout
CodedMonth(s) for April2017:
2017-04
---
May2017 does not match base layout
Can't find CodedMonth column for May2017! Please check manually.
---
June2017 matches base layout
CodedMonth(s) for June2017:
2017-06
---
July2017 matches base layout
CodedMonth(s) for July2017:
2017-07
---
August2017 matches base layout
CodedMonth(s) for August2017:
2017-08
---
September2017 matches base layout
CodedMonth(s) for September2017:
2017-09
---
October2017 matches base layout
CodedMonth(s) for October2017:
2017-10
---
November2017 matches base layout
CodedMonth(s) for November2017:
2017-11
---
December2017 matches base layout
CodedMonth(s) for December2017:
2017-12
---
January2018 matches base layout
CodedMonth(s) for January2018:
2018-01
---
F

So we have three issues: January 2017, May 2017 and May 2018.

January 2017's column names don't match because of the encoding error. We solve that by manually specifying column names. Let's check out May 2017 and May 2018.

In [8]:
print('# May 2017\n')
print(agate.Table.from_csv('temp/May2017.csv'))

print('\n\n# May 2018\n')
print(agate.Table.from_csv('temp/May2018.csv'))

# May 2017

| column              | data_type |
| ------------------- | --------- |
| Complaint           | Text      |
| Coded Month         | Text      |
| Date Occur          | DateTime  |
| Flag-Crime          | Boolean   |
| Flag-Unfounded      | Boolean   |
| Flag-Administrative | Boolean   |
| Count               | Number    |
| Flag Cleanup        | Boolean   |
| Crime               | Number    |
| ILeads Add          | DateTime  |
| ILeads Approve      | Date      |
| District            | Number    |
| Beat                | Text      |
| Description         | Text      |
| ILEADS-Address      | Text      |
| ILEADS-Street       | Text      |
| Neighborhood        | Number    |
| ILeads Asg          | Number    |
| ILeads Type         | Boolean   |
| Location Name       | Text      |
| Location Comment    | Text      |
| CAD-Address         | Text      |
| CAD-Street          | Text      |
| X-Coord             | Number    |
| Y-Coord             | Number    |
| Date Crime Cod

May 2017 includes all the fields we're using (some renamed), but also a bunch of extra stuff. May 2018 looks the same as our base except for the same encoding error.

The encoding error will be taken care of with the explicit column names. But May 2017 will require a different import function.

In [9]:
def import_may_2017_table(file):
    column_names = ['Complaint','CodedMonth','DateOccur','FlagCrime','FlagUnfounded','FlagAdministrative','Count','FlagCleanup','Crime',\
                          'ILEADSAdd','ILEADSApprove','District','Beat','Description','ILEADSAddress','ILEADSStreet','Neighborhood','ILEADSAsg','ILEADSType',\
                          'LocationName','LocationComment','CADAddress','CADStreet','XCoord','YCoord','DateCoded']
                          
    column_types = [text_type,text_type,datetime_type,boolean_type,boolean_type,boolean_type,number_type,boolean_type,text_type,\
                          datetime_type,date_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,boolean_type,\
                          text_type,text_type,text_type,text_type,number_type,number_type,date_type]

    new_table = agate.Table.from_csv(file,column_names,column_types)
    return new_table

## Import files individually, merging with the full table each time

We create a new table first using the column names and types we want. (we're dropping the extra columns from May2017 by explicitly passing `column_names` when merging the table)

In [10]:
column_names = ['Complaint','CodedMonth','DateOccur','FlagCrime','FlagUnfounded','FlagAdministrative','Count','FlagCleanup','Crime','District','Description','ILEADSAddress','ILEADSStreet','Neighborhood','LocationName','LocationComment','CADAddress','CADStreet','XCoord','YCoord']
column_types = [text_type,text_type,datetime_type,boolean_type,boolean_type,boolean_type,number_type,boolean_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,text_type,number_type,number_type]
    
begin = agate.Table([],column_names,column_types)

for file in files:
    filename = 'temp/' + file + '.csv'
    if file != 'May2017':
        cur_table = import_base_table(filename)
        begin = agate.Table.merge([begin,cur_table])
    else:
        cur_table = import_may_2017_table(filename)
        begin = agate.Table.merge([begin,cur_table],column_names=begin.column_names)
        
    print('Importing {}. Adding {} rows. New length: {} rows.'.format(file, len(cur_table), len(begin)))

Importing January2017. Adding 3936 rows. New length: 3936 rows.
Importing February2017. Adding 3548 rows. New length: 7484 rows.
Importing March2017. Adding 3981 rows. New length: 11465 rows.
Importing April2017. Adding 3969 rows. New length: 15434 rows.
Importing May2017. Adding 4548 rows. New length: 19982 rows.
Importing June2017. Adding 4210 rows. New length: 24192 rows.
Importing July2017. Adding 4463 rows. New length: 28655 rows.
Importing August2017. Adding 4503 rows. New length: 33158 rows.
Importing September2017. Adding 3756 rows. New length: 36914 rows.
Importing October2017. Adding 4462 rows. New length: 41376 rows.
Importing November2017. Adding 3847 rows. New length: 45223 rows.
Importing December2017. Adding 3763 rows. New length: 48986 rows.
Importing January2018. Adding 3825 rows. New length: 52811 rows.
Importing February2018. Adding 3185 rows. New length: 55996 rows.
Importing March2018. Adding 3629 rows. New length: 59625 rows.
Importing April2018. Adding 3735 rows.

## Filter to crime occurrences between January and May

We want to do year-to-date comparisons between 2017, 2018 and 2019. So we need to limit to crimes reported to have occurred between January and May.

Why did we not just import January through May? We could have, but crimes that occurred during those months could be reported, marked as unfounded, or reclassified to a different crime later. Note that this does mean that it's not an exactly equal comparison between 2017-2019, because crimes that occurred in 2017 have had two years to be reported or reclassified, when crimes that occurred in 2019 haven't had as much time afterward.

In [11]:
target_dates = begin.where(lambda row: datetime.strptime('2017-01-01', '%Y-%m-%d') <= row['DateOccur'] < datetime.strptime('2017-06-01', '%Y-%m-%d') or\
                           datetime.strptime('2018-01-01', '%Y-%m-%d') <= row['DateOccur'] < datetime.strptime('2018-06-01', '%Y-%m-%d') or\
                           datetime.strptime('2019-01-01', '%Y-%m-%d') <= row['DateOccur'] < datetime.strptime('2019-06-01', '%Y-%m-%d'))

print('Went from {} rows to {} rows.'.format(len(begin),len(target_dates)))

Went from 114255 rows to 56039 rows.


### Check date filtering

In [12]:
earliest = target_dates.select(['DateOccur']).order_by(['DateOccur']).rows[0][0]
latest = target_dates.select(['DateOccur']).order_by(['DateOccur']).rows[-1][0]

earliest_2017 = target_dates.select(['DateOccur']).where(lambda row: row['DateOccur'].year == 2017).order_by(['DateOccur']).rows[1][0]
earliest_2018 = target_dates.select(['DateOccur']).where(lambda row: row['DateOccur'].year == 2018).order_by(['DateOccur']).rows[1][0]
earliest_2019 = target_dates.select(['DateOccur']).where(lambda row: row['DateOccur'].year == 2019).order_by(['DateOccur']).rows[1][0]

latest_2017 = target_dates.select(['DateOccur']).where(lambda row: row['DateOccur'].year == 2017).order_by(['DateOccur']).rows[-1][0]
latest_2018 = target_dates.select(['DateOccur']).where(lambda row: row['DateOccur'].year == 2018).order_by(['DateOccur']).rows[-1][0]
latest_2019 = target_dates.select(['DateOccur']).where(lambda row: row['DateOccur'].year == 2019).order_by(['DateOccur']).rows[-1][0]

print('Overall date range is from {} to {}.'.format(earliest,latest))
print('Dates in 2017 range from {} to {}.'.format(earliest_2017,latest_2017))
print('Dates in 2018 range from {} to {}.'.format(earliest_2018,latest_2018))
print('Dates in 2019 range from {} to {}.'.format(earliest_2019,latest_2019))

Overall date range is from 2017-01-01 00:00:00 to 2019-05-31 23:50:00.
Dates in 2017 range from 2017-01-01 00:01:00 to 2017-05-31 23:45:00.
Dates in 2018 range from 2018-01-01 00:01:00 to 2018-05-31 23:46:00.
Dates in 2019 range from 2019-01-01 00:01:00 to 2019-05-31 23:50:00.


## Reproject state plane coordinates to WGS84

The state plane coordinates used make working with other geodata tough. Because we're going to need to use other shapefiles to filter, we need to reproject these into a more universal coordinate reference system. [see this GitHub repo](https://github.com/kylesykes/stl-crime-data/blob/master/cleaning_stl_crime_data.ipynb)

This step takes a while. I write out the results to a csv file that I can load into a different notebook, so as to not have to repeat this long step later.

In [13]:
state_plane = pyproj.Proj(init='EPSG:26996', preserve_units=True) #East MO State Plane Coord
wgs = pyproj.Proj(proj='latlong', datum='WGS84', ellps='WGS84') #Want WGS84

def convert(x,y):

    x *= Decimal(0.3048)  #Change to meters
    y *= Decimal(0.3048)  #Change to meters
    lng, lat = pyproj.transform(state_plane, wgs, x, y) #Get lng/lat
    return lat,lng

class reprojectCoords(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Text()
    
    def run(self, table):
        new_column = []
        
        for index, row in enumerate(table.rows):
            if index % 100 == 0:
                print('{} of {}'.format(index,len(table)))
            if row['XCoord'] == 0:
                new_column.append(None)
            else:
                new_column.append(convert(row['XCoord'],row['YCoord']))
            
        return new_column

In [14]:
reprojected = target_dates.compute([
    ('coords', reprojectCoords())
])

0 of 56039
100 of 56039
200 of 56039
300 of 56039
400 of 56039
500 of 56039
600 of 56039
700 of 56039
800 of 56039
900 of 56039
1000 of 56039
1100 of 56039
1200 of 56039
1300 of 56039
1400 of 56039
1500 of 56039
1600 of 56039
1700 of 56039
1800 of 56039
1900 of 56039
2000 of 56039
2100 of 56039
2200 of 56039
2300 of 56039
2400 of 56039
2500 of 56039
2600 of 56039
2700 of 56039
2800 of 56039
2900 of 56039
3000 of 56039
3100 of 56039
3200 of 56039
3300 of 56039
3400 of 56039
3500 of 56039
3600 of 56039
3700 of 56039
3800 of 56039
3900 of 56039
4000 of 56039
4100 of 56039
4200 of 56039
4300 of 56039
4400 of 56039
4500 of 56039
4600 of 56039
4700 of 56039
4800 of 56039
4900 of 56039
5000 of 56039
5100 of 56039
5200 of 56039
5300 of 56039
5400 of 56039
5500 of 56039
5600 of 56039
5700 of 56039
5800 of 56039
5900 of 56039
6000 of 56039
6100 of 56039
6200 of 56039
6300 of 56039
6400 of 56039
6500 of 56039
6600 of 56039
6700 of 56039
6800 of 56039
6900 of 56039
7000 of 56039
7100 of 56039
7200

55400 of 56039
55500 of 56039
55600 of 56039
55700 of 56039
55800 of 56039
55900 of 56039
56000 of 56039


## Break out lat/lng into columns

Previous computation created a tuple. We'll want to more easily access it, so let's create columns.

In [15]:
class splitLat(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Number()
    
    def run(self, table):
        new_column = []
        
        for index,row in enumerate(table.rows):
            if index % 100 == 0:
                print('Lat: {} of {}'.format(index,len(table)))            
            if row['coords'] is not None:
                new_column.append(Decimal(row['coords'][0]))
            else:
                new_column.append(None)
                
        return new_column
                
class splitLon(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Number()
    
    def run(self, table):
        new_column = []
        
        for index,row in enumerate(table.rows):
            if index % 100 == 0:
                print('Lon: {} of {}'.format(index,len(table)))
            
            if row['coords'] is not None:
                new_column.append(Decimal(row['coords'][1]))
            else:
                new_column.append(None)
                
        return new_column
                
splitLatLon = reprojected.compute([
    ('lat', splitLat()),
    ('lon', splitLon())
])

Lat: 0 of 56039
Lat: 100 of 56039
Lat: 200 of 56039
Lat: 300 of 56039
Lat: 400 of 56039
Lat: 500 of 56039
Lat: 600 of 56039
Lat: 700 of 56039
Lat: 800 of 56039
Lat: 900 of 56039
Lat: 1000 of 56039
Lat: 1100 of 56039
Lat: 1200 of 56039
Lat: 1300 of 56039
Lat: 1400 of 56039
Lat: 1500 of 56039
Lat: 1600 of 56039
Lat: 1700 of 56039
Lat: 1800 of 56039
Lat: 1900 of 56039
Lat: 2000 of 56039
Lat: 2100 of 56039
Lat: 2200 of 56039
Lat: 2300 of 56039
Lat: 2400 of 56039
Lat: 2500 of 56039
Lat: 2600 of 56039
Lat: 2700 of 56039
Lat: 2800 of 56039
Lat: 2900 of 56039
Lat: 3000 of 56039
Lat: 3100 of 56039
Lat: 3200 of 56039
Lat: 3300 of 56039
Lat: 3400 of 56039
Lat: 3500 of 56039
Lat: 3600 of 56039
Lat: 3700 of 56039
Lat: 3800 of 56039
Lat: 3900 of 56039
Lat: 4000 of 56039
Lat: 4100 of 56039
Lat: 4200 of 56039
Lat: 4300 of 56039
Lat: 4400 of 56039
Lat: 4500 of 56039
Lat: 4600 of 56039
Lat: 4700 of 56039
Lat: 4800 of 56039
Lat: 4900 of 56039
Lat: 5000 of 56039
Lat: 5100 of 56039
Lat: 5200 of 56039
Lat: 

Lon: 24900 of 56039
Lon: 25000 of 56039
Lon: 25100 of 56039
Lon: 25200 of 56039
Lon: 25300 of 56039
Lon: 25400 of 56039
Lon: 25500 of 56039
Lon: 25600 of 56039
Lon: 25700 of 56039
Lon: 25800 of 56039
Lon: 25900 of 56039
Lon: 26000 of 56039
Lon: 26100 of 56039
Lon: 26200 of 56039
Lon: 26300 of 56039
Lon: 26400 of 56039
Lon: 26500 of 56039
Lon: 26600 of 56039
Lon: 26700 of 56039
Lon: 26800 of 56039
Lon: 26900 of 56039
Lon: 27000 of 56039
Lon: 27100 of 56039
Lon: 27200 of 56039
Lon: 27300 of 56039
Lon: 27400 of 56039
Lon: 27500 of 56039
Lon: 27600 of 56039
Lon: 27700 of 56039
Lon: 27800 of 56039
Lon: 27900 of 56039
Lon: 28000 of 56039
Lon: 28100 of 56039
Lon: 28200 of 56039
Lon: 28300 of 56039
Lon: 28400 of 56039
Lon: 28500 of 56039
Lon: 28600 of 56039
Lon: 28700 of 56039
Lon: 28800 of 56039
Lon: 28900 of 56039
Lon: 29000 of 56039
Lon: 29100 of 56039
Lon: 29200 of 56039
Lon: 29300 of 56039
Lon: 29400 of 56039
Lon: 29500 of 56039
Lon: 29600 of 56039
Lon: 29700 of 56039
Lon: 29800 of 56039


## Save to CSV

(don't have to repeat slow reprojection step)

In [16]:
splitLatLon.to_csv('temp/with_lat_lon.csv')